In [32]:
from langchain_classic.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder , ChatPromptTemplate
from langchain_core.messages import HumanMessage , AIMessage
from langchain_groq import ChatGroq
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter
from langchain_classic.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

from dotenv import load_dotenv
load_dotenv()

True

In [42]:
llm = ChatGroq(
    model="openai/gpt-oss-120b",
    temperature=0
)

In [43]:
embeddings =  HuggingFaceEmbeddings(
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
)

In [44]:
loader = PyPDFLoader("../Surface_Treatment_of_Metals.pdf")
texts = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(texts)

vectorstore = FAISS.from_documents(chunks, embeddings)

base_retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

In [45]:
contex_q_sys_prompt = """Given a chat history and the latest user question which might reference context in the chat history ,
formulate a standalone question which can be understood without the chat history . Don't answer
the question , just reformulate it if needed and otherwise return it as is."""

contex_q_prompt = ChatPromptTemplate.from_messages([
    ("system" , contex_q_sys_prompt) , 
    MessagesPlaceholder("chat_history") , 
    ("human" , "{input}")
])

In [46]:
hist_aware_ret = create_history_aware_retriever(
    llm , base_retriever , contex_q_prompt
)

In [47]:
hist_aware_ret

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002125FFB93A0>, search_kwargs={'k': 4}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(

In [48]:
qa_sys_prompt = """You are an assistant for question-answering tasks.
use up the following pieces of retrieved coontext to answer the question.
iff you don't know the answer , just say that you don't know.
se three sentences maximum and keep the answer concise.

context : {context}"""

qa_prompt = ChatPromptTemplate.from_messages([
    ("system" , qa_sys_prompt) , 
    MessagesPlaceholder("chat_history") , 
    ("human" , "{input}")
])

In [49]:
qa_chain = create_stuff_documents_chain(llm , qa_prompt)

In [50]:
chain = create_retrieval_chain (
    hist_aware_ret , 
    qa_chain
)

In [51]:
chat_history = []
result1 = chain.invoke({
    "chat_history" : chat_history , 
    "input" : "What is metal surface treatment"
})
print(result1['answer'])

Metal surface treatment refers to the various processes—such as coating, diffusion (e.g., nitriding), laser heat treatment, or chemical modification—applied to a metal’s exterior to alter its physical, chemical, or mechanical properties. These treatments aim to improve characteristics like corrosion resistance, wear resistance, biocompatibility, and durability. The field encompasses both traditional techniques and advanced methods that are tailored to specific alloy systems.


In [ ]:
chat_history.extend([
    HumanMessage(content="What is metal surface treatment"),
    AIMessage(content=result1['answer'])
])

In [53]:
chat_history

[HumanMessage(content='What is metal surface treatment', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Metal surface treatment refers to the various processes—such as coating, diffusion (e.g., nitriding), laser heat treatment, or chemical modification—applied to a metal’s exterior to alter its physical, chemical, or mechanical properties. These treatments aim to improve characteristics like corrosion resistance, wear resistance, biocompatibility, and durability. The field encompasses both traditional techniques and advanced methods that are tailored to specific alloy systems.', additional_kwargs={}, response_metadata={})]

In [54]:
result2 = chain.invoke({
    "chat_history" : chat_history , 
    "input" : "What it is main types ?"
})
print(result2['answer'])

The main categories of metal surface treatment are (1) coating processes (e.g., ceramic ZrO₂ layers, metallic or polymeric films) that create a protective outer layer, (2) diffusion treatments such as nitriding, carburizing or sulphur‑blocking that alter the surface chemistry by introducing elements into the substrate, and (3) thermal/laser‑based methods (laser heat treatment, laser‑nitriding) that modify microstructure and residual stresses. These approaches are combined or applied separately to enhance corrosion resistance, wear resistance, biocompatibility and overall durability.


The main categories of metal surface treatment are (1) coating processes (e.g., ceramic ZrO₂ layers, metallic or polymeric films) that create a protective outer layer, (2) diffusion treatments such as nitriding, carburizing or sulphur‑blocking that alter the surface chemistry by introducing elements into the substrate, and (3) thermal/laser‑based methods (laser heat treatment, laser‑nitriding) that modify microstructure and residual stresses. These approaches are combined or applied separately to enhance corrosion resistance, wear resistance, biocompatibility and overall durability.

In [55]:
chat_history.extend([
    HumanMessage(content="What it is main types ?"),
    AIMessage(content=result2['answer'])
])